# Preprocessing
Below we are going to preprocess the IMDB dataset and clean out not useful data

In [1]:
#load libraries
import pandas as pd
import numpy as np

In [2]:
#basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
#ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'
#akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'


In [3]:
#load in dataset this is the basics one first
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)



In [4]:
#display the info 
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9719896 entries, 0 to 9719895
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 667.4+ MB


In [5]:
#check for missing data this check shows only primary title original title and genres are missing values but \N was used on this data and must be replaced to see tru missing values
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            11
dtype: int64

In [6]:
#premanantly replace \N with nan
basics.replace({'\\N':np.nan}, inplace = True)

In [7]:
#check for missing data again
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1315440
endYear           9615510
runtimeMinutes    6863616
genres             438181
dtype: int64

In [8]:
#check for missing data again
basics['runtimeMinutes'].value_counts()

30      217346
22      151432
60      149335
44       75376
15       74956
         ...  
1120         1
1452         1
1554         1
850          1
2088         1
Name: runtimeMinutes, Length: 885, dtype: int64

In [9]:
#no duplicates
basics.duplicated().sum()

0

In [10]:
#check how many values are missing in runtimeminutes columns
basics['runtimeMinutes'].isnull().value_counts()

True     6863616
False    2856280
Name: runtimeMinutes, dtype: int64

In [11]:
#drop them
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [12]:
#check again and theyre gone
basics['runtimeMinutes'].isnull().value_counts()

False    2856280
Name: runtimeMinutes, dtype: int64

In [13]:
#check how many rows have null values
basics['genres'].isnull().value_counts()

False    2779892
True       76388
Name: genres, dtype: int64

In [14]:
#drop them
basics.dropna(subset=['genres'], inplace=True)

In [15]:
#check they were dropped
basics['genres'].isnull().value_counts()

False    2779892
Name: genres, dtype: int64

In [16]:
#check that we only include movie titles 
basics['titleType'].value_counts()

tvEpisode       1399850
short            595919
movie            379665
video            179573
tvMovie           91134
tvSeries          89846
tvSpecial         17870
tvMiniSeries      16998
tvShort            8720
videoGame           317
Name: titleType, dtype: int64

In [17]:
#make a filter to filter out movies
movie_filter = basics['titleType'] == 'movie'


In [18]:
#apply filter to dataset and make it permanant
basics = basics[movie_filter]

In [19]:
#check if it changed
basics['titleType'].value_counts()

movie    379665
Name: titleType, dtype: int64

In [20]:
#filter out only movies between 2000 and 2022
basics['startYear'].value_counts()

2017    14357
2018    14303
2019    14029
2016    13937
2015    13453
        ...  
1899        1
1904        1
1897        1
1896        1
1894        1
Name: startYear, Length: 129, dtype: int64

In [21]:
#make a filter for years below 2000
year_filter = basics['startYear'] >'1999'

In [22]:
#make a filter for years above 2022
year_filter2 = basics['startYear']<='2022'

In [23]:
#combine both filters and apply them to the df
basics= basics.loc[year_filter & year_filter2,:]

In [24]:
#check results
basics['startYear'].value_counts()

2017    14357
2018    14303
2019    14029
2016    13937
2015    13453
2014    13079
2022    12536
2013    12366
2021    12279
2012    11612
2020    11528
2011    10760
2010    10193
2009     9335
2008     8139
2007     6950
2006     6500
2005     5814
2004     5186
2003     4577
2002     4127
2001     3853
2000     3633
Name: startYear, dtype: int64

In [25]:
#make a filter to remove the documantary type movies
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [26]:
#check results
basics['genres'].value_counts()

Drama                         35918
Comedy                        13420
Comedy,Drama                   6449
Horror                         5770
Drama,Romance                  4299
                              ...  
Action,Animation,Game-Show        1
Family,Musical,Sport              1
Horror,Music,Mystery              1
Comedy,History,Mystery            1
Crime,Fantasy,Sci-Fi              1
Name: genres, Length: 969, dtype: int64

In [27]:
#load akas dataframe
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [50]:
#check it out
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1420891
types               448412
attributes         1378501
isOriginalTitle       1345
dtype: int64

In [28]:
#replace any \N values with np.nan
akas.replace({'\\N':np.nan}, inplace = True)

In [48]:
#check resuts for more hidden missing values
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1420891
types               448412
attributes         1378501
isOriginalTitle       1345
dtype: int64

In [40]:
#filter out the US only moves 
us_filter = akas['region'] =='US'

In [41]:
#apply it to the dataset
akas = akas[us_filter]

In [42]:
#check results
akas['region'].value_counts()

US    1424761
Name: region, dtype: int64

In [43]:
#apply this to the basics df
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34803       True
61116       True
67669       True
77964      False
86801       True
           ...  
9719569     True
9719578     True
9719617    False
9719662     True
9719746    False
Name: tconst, Length: 146409, dtype: bool

In [44]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9719034,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9719429,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9719569,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9719578,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [31]:
#Load rating dataset
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [32]:
#no missisng values
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [45]:
#filter only us movies using akas df
keepers2 =ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1293884    False
1293885    False
1293886    False
1293887    False
1293888    False
Name: tconst, Length: 1293889, dtype: bool

In [46]:
#check results
ratings= ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,263
4,tt0000005,6.2,2603
5,tt0000006,5.1,178
6,tt0000007,5.4,817
...,...,...,...
1293854,tt9916200,8.1,226
1293855,tt9916204,8.2,258
1293862,tt9916348,8.3,18
1293863,tt9916362,6.4,5247


In [51]:
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

In [53]:
basics.to_csv('basics.csv')

In [54]:
akas.to_csv('akas.csv')

In [55]:
ratings.to_csv('ratings.csv')

In [56]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [58]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [59]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [57]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [60]:
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [61]:
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1966
1,tt0000002,5.8,263
2,tt0000005,6.2,2603
3,tt0000006,5.1,178
4,tt0000007,5.4,817
